In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
font = {'family' : 'normal',
        'size'   : 22}

In [ ]:
matplotlib.rc('font', **font)

In [ ]:
path_geral = '/content/drive/MyDrive/leaves/'
path_gold = '/content/drive/MyDrive/leaves/padrao-ouro/'

In [ ]:
img  = cv.imread(path_geral+'002.jpg')

In [ ]:
img2 = cv.cvtColor(img, cv.COLOR_BGR2RGB)

In [ ]:
plt.imshow(img2)

In [ ]:
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

In [ ]:
img_mask = cv.imread(path_gold+'002_mask.png')

In [ ]:
plt.imshow(img_mask)

In [ ]:
gray = cv.cvtColor(img2,cv.COLOR_RGB2GRAY)
ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)

In [ ]:
plt.imshow(thresh)

In [ ]:
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)

In [ ]:
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)

In [ ]:
sure_bg = cv.dilate(opening,kernel,iterations=3)

In [ ]:
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)

In [ ]:
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg,sure_fg)

In [ ]:
ret, markers = cv.connectedComponents(sure_fg)

In [ ]:
markers = markers+1

In [ ]:
markers = markers+1
markers[unknown==255] = 0
markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,0]
plt.imshow(markers)

In [ ]:
markers[unknown==255] = 0

In [ ]:
markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,0]

In [ ]:
plt.imshow(markers)

In [ ]:
plt.imshow(img)

axs[0,0].imshow(img2)
axs[0,1].imshow(img_mask)
axs[0,2].imshow(markers)
fig.suptitle('Imagem original x Mascara Padrão Ouro x Filtro watershed')

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(20,15))
fig.suptitle('Imagem original x mask padrão ouro x mask watershed')
ax[0].imshow(img2)
ax[0].set_title('Imagem original')
ax[1].imshow(markers)
ax[1].set_title('mask watershed')


In [ ]:
path_mascara = '/content/drive/MyDrive/leaves/mascara_watershed/'

In [ ]:
def plot_and_save(img,maskwater,name_img):
    fig,ax = plt.subplots(1,2, figsize=(10,15))
    fig.suptitle('Imagem original x mask padrão ouro x mask watershed')
    ax[0].imshow(img)
    ax[0].set_title('Imagem original')
    ax[1].imshow(maskwater)
    ax[1].set_title('mask watershed')

    plt.savefig(path_mascara+name_img+'.jpg', bbox_inches='tight',pad_inches = 0)
    plt.imsave(path_mascara+name_img+'filtro'+'.jpg', maskwater)

In [ ]:
plot_and_save(img2,markers,'img002')

In [ ]:

def im_write(img2,name_img):
    img  = cv.imread(img2)
    img2 = cv.cvtColor(img,cv.COLOR_BGR2RGB)
    gray = cv.cvtColor(img2,cv.COLOR_RGB2GRAY) # escala de cinza
    ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU) #pegando threshold
    kernel = np.ones((3,3),np.uint8) # criando o kernel 3 x 3
    opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2) # aplicando tecnicas de erosão e posteriomente dilatação
    dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5) #
    sure_bg = cv.dilate(opening,kernel,iterations=3)
    ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg,sure_fg)
    ret, markers = cv.connectedComponents(sure_fg)
    markers = markers+1
    markers[unknown==255] = 0
    markers = cv.watershed(img,markers)
    img[markers == -1] = [255,0,0]
    #plt.imshow(markers)
    # regex
    #cv.imwrite(path_mascara+name_img+'.jpg',markers)
    plot_and_save(img2,markers,name_img)

In [ ]:
import re

In [ ]:
import glob

In [ ]:
imgs = [im for im in glob.glob(path_geral+'*.jpg')]

In [ ]:
regex = r"/([0-9]+).jpg"

In [ ]:
names_imgs = []

for img in imgs:
    matches = re.search(regex, img)
    print(matches.group(1))
    names_imgs.append(matches.group(1))

In [ ]:
imgs

In [ ]:
for img, name in zip(imgs,names_imgs):
    im_write(img,name)